In [1]:
!pip install snowflake-connector-python
!pip install tldextract
!pip install py2neo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 kB 12.4 MB/s eta 0:00:00


In [ ]:
df['ColumnA'] = df[df.columns[1:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

In [ ]:
import requests
import pandas as pd
import numpy as np
import snowflake.connector
import os
from py2neo import Graph
from py2neo.bulk import merge_nodes, create_relationships, merge_relationships
from urllib.parse import urlparse

#[Environment Configs]
client_id = "d0afa05e7ab642acb0497f018b601b86"
client_secret = "ikdU3eOtXRmMR9oWlEnP0W0jAVZYNMBF2FUwsuwtHkZ37Oh4IZZkso3tFdsbfdmO"  
os.environ["HTTP_PROXY"] = "http://d161560-001.dc.gs.com:8899"
os.environ["HTTPS_PROXY"] = "http://d161560-001.dc.gs.com:8899"

#[Credentials]
host = "xplore.graph-x.site.gs.com"
port = "7687"
username = "neo4j"
password = "xplorer"
graph = Graph(host=host, port=port, auth=(username, password))
BATCHES = 100

### Libraries & Shared Code

In [ ]:
def get_oauth_tokens(client_id: str, client_secret: str):
    _IDFS_URL = "https://idfs.gs.com/as/token.oauth2"
    response = requests.post(
        url = _IDFS_URL,
        params = {"access_token_manager_id": "JwtSnowflake"},  # This must be set to JwtSnowflake to generate Snowflake-compatible OAuth Token
        data = {"grant_type": "client_credentials", "scope": "SESSION:ROLE-ANY"},
        auth = (client_id, client_secret),
    )
    response = response.json()
    if "access_token" not in response.keys():
        raise PermissionError(f"Not authorized to access OAuth Tokens: {response}")
    return response["access_token"]
  
def connect_snowflake(user: str, database: str, token: str) -> snowflake.connector.cursor:
    """Connect to Snowflake and return an authorized cursor. Below uses Catalyst connections
   
    :param user: Client ID from OAuth2 Authentication
    :param database: Name of database to connect to
    :param token: Generated OAuth Token. Please either generate and save to a tmp file in a separate thread OR use subprocess module to call the underlying subsystem and capture the output
    :return: Snowflake Cursor
    """
    snow_connector = snowflake.connector.connect(
        user=user,
        authenticator="oauth",
        account="sfamdprvtawseast1d01.goldman.us-east-1.aws.privatelink",
        role="AMD_CATALYST_DATA_RW",
        warehouse="AMD_CATALYST_RW",
        database=database,
        ocsp_fail_open=False,
        token=token,
    )
    cursor = snow_connector.cursor()
    return cursor
 
def disconnect(cursor: snowflake.connector.cursor) -> None:
    try:
        cursor.close()
    except:
        raise Exception("Failed to close cursor")   
        
oauth_token = get_oauth_tokens(client_id, client_secret)
cursor = connect_snowflake(client_id, "AMD_CATALYST_DB", oauth_token)

### SourceScrub

In [ ]:
sourcescrub_organizations_df = cursor.execute('SELECT ID, NAME, DOMAIN, LINKEDIN, TWITTER, FACEBOOK, CRUNCHBASE, FOUNDINGYEAR, CITY, STATE, COUNTRY, SPECIALTIES, TOTALAMOUNTINVESTED FROM L1.SOURCESCRUB_COMPANY WHERE DOMAIN IS NOT NULL OR LINKEDIN IS NOT NULL OR TWITTER IS NOT NULL OR FACEBOOK IS NOT NULL OR CRUNCHBASE IS NOT NULL').fetch_pandas_all()
display(sourcescrub_organizations_df.head(10))

,ID,NAME,DOMAIN,LINKEDIN,TWITTER,FACEBOOK,CRUNCHBASE,FOUNDINGYEAR,CITY,STATE,COUNTRY,SPECIALTIES,TOTALAMOUNTINVESTED
0,2475887,Spacehuhn Technologies UG,spacehuhn.com,None,https://twitter.com/spacehuhntech,https://www.facebook.com/spacehuhn-technologie...,None,2020.0,Bingen am Rhein,Rheinland-Pfalz,Germany,Information Technology,NaN
1,2475885,"Public Invention, Inc.",pubinv.org,https://www.linkedin.com/company/public-invention,https://twitter.com/pubinvention,https://www.facebook.com/publicinvention,https://www.crunchbase.com/organization/public...,2015.0,Austin,Texas,United States,Non-profit,NaN
2,2475884,"8E6, Ltd.",8e6.ca,None,None,None,None,2020.0,Richmond Hill Southwest,Ontario,Canada,Pain Relieving Towellete,NaN
3,2475873,"Waterotor Energy Technologies, Inc.",waterotor.com,https://www.linkedin.com/company/waterotor-ene...,https://twitter.com/waterotor1,https://www.facebook.com/waterotor,https://www.crunchbase.com/organization/watero...,2011.0,Ottawa,Ontario,Canada,"renewable energy, hydrokinetic energy, Hydro P...",NaN
4,2475872,UnGUN Institute,unguninstitute.com,None,None,https://www.facebook.com/disarmingtrauma,None,2017.0,St. Louis,Missouri,United States,Community Service,NaN
5,2475871,"Georgia Highway Contractors Association, Inc. ...",gahca.com,None,None,None,None,1937.0,Alpharetta,Georgia,United States,"Transportation, Construction, Roadways, State",NaN
6,2475869,Academic Center for Coherent Intelligence (ACCI),acci.center,https://www.linkedin.com/company/academic-cent...,https://twitter.com/igor_valdanilov,https://www.facebook.com/academiccenterforcohe...,None,2018.0,Riga,Riga,Latvia,social interactions,NaN
7,2475868,"CultureX, Inc.",culturex.com,https://www.linkedin.com/company/culturex,https://twitter.com/culturexinsight,https://www.facebook.com/culturex,None,2019.0,Cambridge,Massachusetts,United States,Corporate Culture and Artificial Intelligence,NaN
8,2475866,Wolk After Sales Experts GmbH,wolk-aftersales.com,https://www.linkedin.com/company/wolk-after-sa...,https://twitter.com/wolkaftersales,https://www.facebook.com/wolkaftersales,None,2009.0,Bergisch Gladbach,Nordrhein-Westfalen,Germany,"Automotive Aftermarket, Automotive Market Rese...",NaN
9,2475865,"Rockford Construction, Inc.",rockfordconstruction.net,None,None,None,None,2009.0,Columbus,Ohio,United States,General Construction,NaN


        ID                                               NAME  \
0  2475887                          Spacehuhn Technologies UG   
1  2475885                             Public Invention, Inc.   
2  2475884                                          8E6, Ltd.   
3  2475873                Waterotor Energy Technologies, Inc.   
4  2475872                                    UnGUN Institute   
5  2475871  Georgia Highway Contractors Association, Inc. ...   
6  2475869   Academic Center for Coherent Intelligence (ACCI)   
7  2475868                                     CultureX, Inc.   
8  2475866                      Wolk After Sales Experts GmbH   
9  2475865                        Rockford Construction, Inc.   

                     DOMAIN  \
0             spacehuhn.com   
1                pubinv.org   
2                    8e6.ca   
3             waterotor.com   
4        unguninstitute.com   
5                 gahca.com   
6               acci.center   
7              culturex.com   
8    

In [ ]:
sourcescrub_organizations_df.rename(columns={'ID': 'SOURCE_ID', 'FOUNDINGYEAR': 'TIME_OF_FOUNDING', 'TOTALAMOUNTINVESTED': 'TOTAL_AMOUNT_INVESTED'}, inplace=True)
sourcescrub_organizations_df['SOURCE_ID'] = sourcescrub_organizations_df['SOURCE_ID'].astype(str)
sourcescrub_organizations_df['TIME_OF_FOUNDING'] = sourcescrub_organizations_df['TIME_OF_FOUNDING'].fillna(0).astype('int').replace(0, np.nan)
sourcescrub_organizations_df['SOURCE_ID'] = 'sourcescrub-' + sourcescrub_organizations_df['SOURCE_ID']

sourcescrub_organizations_df['LINKEDIN'] = sourcescrub_organizations_df['LINKEDIN'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))
sourcescrub_organizations_df['TWITTER'] = sourcescrub_organizations_df['TWITTER'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))
sourcescrub_organizations_df['FACEBOOK'] = sourcescrub_organizations_df['FACEBOOK'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))
sourcescrub_organizations_df['CRUNCHBASE'] = sourcescrub_organizations_df['CRUNCHBASE'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))

sourcescrub_organizations_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [ ]:
sourcescrub_organizations_df.head()

,SOURCE_ID,NAME,DOMAIN,LINKEDIN,TWITTER,FACEBOOK,CRUNCHBASE,TIME_OF_FOUNDING,CITY,STATE,COUNTRY,SPECIALTIES,TOTAL_AMOUNT_INVESTED
0,sourcescrub-2475887,Spacehuhn Technologies UG,spacehuhn.com,NaN,twitter.com/spacehuhntech,facebook.com/spacehuhn-technologies-1084143915...,NaN,2020.0,Bingen am Rhein,Rheinland-Pfalz,Germany,Information Technology,NaN
1,sourcescrub-2475885,"Public Invention, Inc.",pubinv.org,linkedin.com/company/public-invention,twitter.com/pubinvention,facebook.com/publicinvention,crunchbase.com/organization/public-invention,2015.0,Austin,Texas,United States,Non-profit,NaN
2,sourcescrub-2475884,"8E6, Ltd.",8e6.ca,NaN,NaN,NaN,NaN,2020.0,Richmond Hill Southwest,Ontario,Canada,Pain Relieving Towellete,NaN
3,sourcescrub-2475873,"Waterotor Energy Technologies, Inc.",waterotor.com,linkedin.com/company/waterotor-energy-technolo...,twitter.com/waterotor1,facebook.com/waterotor,crunchbase.com/organization/waterotor-energy-t...,2011.0,Ottawa,Ontario,Canada,"renewable energy, hydrokinetic energy, Hydro P...",NaN
4,sourcescrub-2475872,UnGUN Institute,unguninstitute.com,NaN,NaN,facebook.com/disarmingtrauma,NaN,2017.0,St. Louis,Missouri,United States,Community Service,NaN


             SOURCE_ID                                 NAME  \
0  sourcescrub-2475887            Spacehuhn Technologies UG   
1  sourcescrub-2475885               Public Invention, Inc.   
2  sourcescrub-2475884                            8E6, Ltd.   
3  sourcescrub-2475873  Waterotor Energy Technologies, Inc.   
4  sourcescrub-2475872                      UnGUN Institute   

               DOMAIN                                           LINKEDIN  \
0       spacehuhn.com                                                NaN   
1          pubinv.org              linkedin.com/company/public-invention   
2              8e6.ca                                                NaN   
3       waterotor.com  linkedin.com/company/waterotor-energy-technolo...   
4  unguninstitute.com                                                NaN   

                     TWITTER  \
0  twitter.com/spacehuhntech   
1   twitter.com/pubinvention   
2                        NaN   
3     twitter.com/waterotor1   
4   

### Ingesting Common SourceScrub Identifiers into Graph

In [ ]:
sourcescrub_organizations_df_website = np.array_split(sourcescrub_organizations_df[sourcescrub_organizations_df['DOMAIN'].isna() == False][['DOMAIN']].rename(columns={'DOMAIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_organizations_df_website:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "DomainURL"), "URL"),
    )

sourcescrub_organizations_df_linkedin = np.array_split(sourcescrub_organizations_df[sourcescrub_organizations_df['LINKEDIN'].isna() == False][['LINKEDIN']].rename(columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_organizations_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "LinkedInURL"), "URL"),
    )
    
sourcescrub_organizations_df_twitter = np.array_split(sourcescrub_organizations_df[sourcescrub_organizations_df['TWITTER'].isna() == False][['TWITTER']].rename(columns={'TWITTER': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_organizations_df_twitter:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "TwitterURL"), "URL"),
    )

sourcescrub_organizations_df_facebook =  np.array_split(sourcescrub_organizations_df[sourcescrub_organizations_df['FACEBOOK'].isna() == False][['FACEBOOK']].rename(columns={'FACEBOOK': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_organizations_df_facebook:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "FacebookURL"), "URL"),
    )

sourcescrub_organizations_df_crunchbase =  np.array_split(sourcescrub_organizations_df[sourcescrub_organizations_df['CRUNCHBASE'].isna() == False][['CRUNCHBASE']].rename(columns={'CRUNCHBASE': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_organizations_df_crunchbase:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "CrunchbaseURL"), "URL"),
    )

### Ingesting SourceScrub Companies into the Graph

In [ ]:
sourcescrub_organizations = np.array_split([{k:v for k, v in x.items() if v == v} for x in sourcescrub_organizations_df.to_dict('records')], BATCHES)
for chunk_df in sourcescrub_organizations:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("Company", "SourcescrubCompany"), "SOURCE_ID"),
    )

In [ ]:
for identifier in ['DOMAIN', 'LINKEDIN', 'TWITTER', 'FACEBOOK', 'CRUNCHBASE']:
    sourcescrub_identifier_relationships = sourcescrub_organizations_df.loc[sourcescrub_organizations_df[identifier] != np.nan][['SOURCE_ID', identifier]]
    sourcescrub_identifier_relationships.insert(1, 'WEIGHT', [[0.5]] * len(sourcescrub_identifier_relationships))

    df_list = np.array_split(sourcescrub_identifier_relationships, BATCHES)
    for chunk_df in df_list:
        merge_relationships(
            graph.auto(),
            chunk_df.values.tolist(),
            merge_key="LINKED_TO",
            start_node_key=("Company", "SOURCE_ID"),
            end_node_key=("URLIdentifier", "URL"),
            keys=["WEIGHT"],
        )

### CrunchBase

In [ ]:
crunchbase_organizations_df = cursor.execute('SELECT CRUNCHBASE_ORGANIZATION_UUID, ORGANIZATION_CITY, FACEBOOK_URL, DOMAIN, TWITTER_URL, FUNDING_ROUND_TYPE, FOUNDED_ON, EMPLOYEE_COUNT, CRUNCHBASE_GROUPS, ORGANIZATION_STATE_CODE, ORGANIZATION_NAME, LINKEDIN_URL, ORGANIZATION_COUNTRY_CODE, CRUNCHBASE_URL FROM L2_NEW.CRUNCHBASE_ORGANIZATIONS WHERE DOMAIN IS NOT NULL OR TWITTER_URL IS NOT NULL OR FACEBOOK_URL IS NOT NULL OR LINKEDIN_URL IS NOT NULL OR CRUNCHBASE_URL IS NOT NULL').fetch_pandas_all()
crunchbase_organizations_df.rename(columns={
    'CRUNCHBASE_ORGANIZATION_UUID': 'SOURCE_ID', 
    'TWITTER_URL': 'TWITTER', 
    'FACEBOOK_URL': 'FACEBOOK',
    'LINKEDIN_URL': 'LINKEDIN',
    'CRUNCHBASE_URL': 'CRUNCHBASE', 
    'FOUNDED_ON': 'TIME_OF_FOUNDING',
    'ORGANIZATION_CITY': 'CITY',
    'ORGANIZATION_STATE_CODE': 'STATE',
    'ORGANIZATION_COUNTRY_CODE': 'COUNTRY', 
    'ORGANIZATION_NAME': 'NAME', 
    'LINKEDIN_URL': 'LINKEDIN'
}, inplace=True)

crunchbase_organizations_df.replace({None: ''}, inplace=True)

crunchbase_organizations_df['LINKEDIN'] = crunchbase_organizations_df['LINKEDIN'].apply(lambda x: (str(urlparse(x.replace('[', '').replace(']', '')).netloc or '') + str(urlparse(x.replace('[', '').replace(']', '')).path or '')).lstrip('www.'))
crunchbase_organizations_df['TWITTER'] = crunchbase_organizations_df['TWITTER'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))
crunchbase_organizations_df['FACEBOOK'] = crunchbase_organizations_df['FACEBOOK'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))
crunchbase_organizations_df['CRUNCHBASE'] = crunchbase_organizations_df['CRUNCHBASE'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))

crunchbase_organizations_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
display(crunchbase_organizations_df.head())

,SOURCE_ID,CITY,FACEBOOK,DOMAIN,TWITTER,FUNDING_ROUND_TYPE,TIME_OF_FOUNDING,EMPLOYEE_COUNT,CRUNCHBASE_GROUPS,STATE,NAME,LINKEDIN,COUNTRY,CRUNCHBASE
0,crunchbase-b883e57f-5a42-4cce-81f3-b70ad9482052,New York,NaN,infinity.xyz,twitter.com/infinitydotxyz,NaN,NaN,unknown,"Financial Services,Lending and Investments,Pay...",NY,Infinity,NaN,USA,crunchbase.com/organization/infinity-2052
1,crunchbase-57eeadb3-5173-4fd9-a9f8-dec1ad472417,New York,facebook.com/pages/category/Insurance-company/...,skyviewagency.com,NaN,NaN,NaN,1-10,Financial Services,NY,SkyView Agency,linkedin.com/company/skyview-agency,USA,crunchbase.com/organization/skyview-agency
2,crunchbase-f81e2ff9-6a87-4629-8a7e-230855a0dd30,Toshima,NaN,muio.co.jp,NaN,NaN,1991-02-01,11-50,"Hardware,Information Technology,Internet Servi...",NaN,Muio Creative,NaN,JPN,crunchbase.com/organization/muio-creative
3,crunchbase-ebff3a2b-1b34-4bc8-8b46-5640352c99c0,Suva,facebook.com/vititalk,vititalk.com,twitter.com/viti_talk,NaN,NaN,unknown,"Hardware,Messaging and Telecommunications",NaN,Viti Talk Telecommunications,NaN,FJI,crunchbase.com/organization/viti-talk-telecomm...
4,crunchbase-2774d086-6884-4d8b-9968-58181ed4e104,Vienna,NaN,membershipcards.com,NaN,NaN,1995-01-01,11-50,"Content and Publishing,Design,Media and Entert...",NaN,Membership Cards Only,linkedin.com/company/membership-cards-only-llc,AUT,crunchbase.com/organization/membership-cards-only


                                         SOURCE_ID      CITY  \
0  crunchbase-b883e57f-5a42-4cce-81f3-b70ad9482052  New York   
1  crunchbase-57eeadb3-5173-4fd9-a9f8-dec1ad472417  New York   
2  crunchbase-f81e2ff9-6a87-4629-8a7e-230855a0dd30   Toshima   
3  crunchbase-ebff3a2b-1b34-4bc8-8b46-5640352c99c0      Suva   
4  crunchbase-2774d086-6884-4d8b-9968-58181ed4e104    Vienna   

                                            FACEBOOK               DOMAIN  \
0                                                NaN         infinity.xyz   
1  facebook.com/pages/category/Insurance-company/...    skyviewagency.com   
2                                                NaN           muio.co.jp   
3                              facebook.com/vititalk         vititalk.com   
4                                                NaN  membershipcards.com   

                      TWITTER FUNDING_ROUND_TYPE TIME_OF_FOUNDING  \
0  twitter.com/infinitydotxyz                NaN              NaN   
1             

In [ ]:
crunchbase_organizations_df_website = np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['DOMAIN'].isna() == False][['DOMAIN']].rename(columns={'DOMAIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_website:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "DomainURL"), "URL"),
    )

crunchbase_organizations_df_linkedin = np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['LINKEDIN'].isna() == False][['LINKEDIN']].rename(columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "LinkedInURL"), "URL"),
    )
    
crunchbase_organizations_df_twitter = np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['TWITTER'].isna() == False][['TWITTER']].rename(columns={'TWITTER': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_twitter:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "TwitterURL"), "URL"),
    )

crunchbase_organizations_df_facebook =  np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['FACEBOOK'].isna() == False][['FACEBOOK']].rename(columns={'FACEBOOK': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_facebook:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "FacebookURL"), "URL"),
    )

crunchbase_organizations_df_crunchbase =  np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['CRUNCHBASE'].isna() == False][['CRUNCHBASE']].rename(columns={'CRUNCHBASE': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_crunchbase:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "CrunchbaseURL"), "URL"),
    )

In [ ]:
crunchbase_organizations = np.array_split([{k:v for k, v in x.items() if v == v} for x in crunchbase_organizations_df.to_dict('records')], BATCHES)
for chunk_df in crunchbase_organizations:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("Company", "CrunchbaseCompany"), "SOURCE_ID"),
    )

In [ ]:
for identifier in ['DOMAIN', 'LINKEDIN', 'TWITTER', 'FACEBOOK', 'CRUNCHBASE']:
    crunchbase_identifier_relationships = crunchbase_organizations_df.loc[crunchbase_organizations_df[identifier] != np.nan][['SOURCE_ID', identifier]]
    crunchbase_identifier_relationships.insert(1, 'WEIGHT', [[0.5]] * len(crunchbase_identifier_relationships))

    df_list = np.array_split(crunchbase_identifier_relationships, BATCHES)
    for chunk_df in df_list:
        merge_relationships(
            graph.auto(),
            chunk_df.values.tolist(),
            merge_key="LINKED_TO",
            start_node_key=("Company", "SOURCE_ID"),
            end_node_key=("URLIdentifier", "URL"),
            keys=["WEIGHT"],
        )